In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale
import matplotlib
from matplotlib import pylab, gridspec, pyplot as plt
from sklearn.metrics import f1_score
import xgboost
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

In [2]:
df = pd.read_csv('train.csv', sep='\t', low_memory=False)

In [3]:
df.head()

,pdb_chain,DSSR,xray,resol,chainlen,protein,alpham2,betam2,gammam2,deltam2,...,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3,mg
0,1b23.cif1_R,R.G.1.,1,2.6,74,1,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
1,1b23.cif1_R,R.G.1.,1,2.6,74,1,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
2,1b23.cif1_R,R.G.1.,1,2.6,74,1,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
3,1b23.cif1_R,R.G.1.,1,2.6,74,1,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
4,1b23.cif1_R,R.G.1.,1,2.6,74,1,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0


In [3]:
df.dropna(inplace=True)

In [87]:
describe = df.describe(include='all')

In [142]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):
    print(df)

          pdb_chain     DSSR  xray  resol  chainlen  protein  alpham2  betam2  \
35      1b23.cif1_R   R.G.4.     1   2.60        74        1    -60.0   177.7   
36      1b23.cif1_R   R.G.4.     1   2.60        74        1    -60.0   177.7   
37      1b23.cif1_R   R.G.4.     1   2.60        74        1    -60.0   177.7   
38      1b23.cif1_R   R.G.4.     1   2.60        74        1    -60.0   177.7   
39      1b23.cif1_R   R.G.4.     1   2.60        74        1    -60.0   177.7   
...             ...      ...   ...    ...       ...      ...      ...     ...   
237869  5up6.cif1_R  R.G.17.     0   3.74        20        1    -74.3   155.1   
237870  5up6.cif1_R  R.G.17.     0   3.74        20        1    -74.3   155.1   
237871  5up6.cif1_R  R.G.17.     0   3.74        20        1    -74.3   155.1   
237872  5up6.cif1_R  R.G.17.     0   3.74        20        1    -74.3   155.1   
237873  5up6.cif1_R  R.G.17.     0   3.74        20        1    -74.3   155.1   

        gammam2  deltam2  e

In [5]:
def number_encode_features(init_df, col_index):
    result = init_df.copy()
    encoders = {}
    for column in col_index:
        if result.dtypes[column] == np.object:
            encoders[column] = LabelEncoder()
            result[column] = encoders[column].fit_transform(result[column])
    return result, encoders

# col_index = np.ravel(np.argwhere(describe.iloc[1, :] < 4))

In [6]:
col_index = ['pdb_chain' , 'DSSR']
encoded_data, encoders = number_encode_features(df, col_index)
encoded_data.head()

,pdb_chain,DSSR,xray,resol,chainlen,protein,alpham2,betam2,gammam2,deltam2,...,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3,mg
35,0,9577,1,2.6,74,1,-60.0,177.7,50.4,84.8,...,0,0,0,0,0,0,1,0,0,0
36,0,9577,1,2.6,74,1,-60.0,177.7,50.4,84.8,...,0,0,0,0,0,0,0,1,0,0
37,0,9577,1,2.6,74,1,-60.0,177.7,50.4,84.8,...,0,0,0,0,1,0,0,0,0,0
38,0,9577,1,2.6,74,1,-60.0,177.7,50.4,84.8,...,0,0,0,1,0,0,0,0,0,0
39,0,9577,1,2.6,74,1,-60.0,177.7,50.4,84.8,...,0,1,0,0,0,0,0,0,0,0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(encoded_data.iloc[:, 0 : 385], encoded_data.iloc[:, -1], test_size=0.0, random_state=42)

In [13]:
X_train = scale(X_train)

In [14]:
X_test = scale(X_test)

ValueError: Found array with 0 sample(s) (shape=(0, 385)) while a minimum of 1 is required by the scale function.

In [15]:
X_train.shape, X_test.shape

((209043, 385), (0, 385))

In [203]:
def grid_plot(x, y, x_label, title, y_label):
    plt.figure(figsize=(12, 6))
    plt.grid(True)
    plt.plot(x, y, 'go-')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

In [120]:
model = KNeighborsClassifier(n_jobs=-1)

In [144]:
from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=1, pos_label='1')

In [145]:
grid = {'n_neighbors': np.array(np.arange(3, 11, 2), dtype='int')}
gs = GridSearchCV(model, grid, cv=5, n_jobs=-1, scoring='f1', verbose=2)

In [146]:
grid

{'n_neighbors': array([3, 5, 7, 9])}

In [147]:
gs.fit(X_train, y_train)
print(gs.best_params_, gs.best_score_)


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] n_neighbors=3 ...................................................
[CV] n_neighbors=3 ...................................................
[CV] n_neighbors=3 ...................................................
[CV] n_neighbors=3 ...................................................
[CV] n_neighbors=3 ...................................................
[CV] n_neighbors=5 ...................................................
[CV] n_neighbors=5 ...................................................
[CV] n_neighbors=5 ...................................................


KeyboardInterrupt: 

In [ ]:
grid_plot(grid['n_neighbors'], gs.cv_results_['mean_test_score'], 'n_neighbors', 'KNeighborsClassifier', y_label='f1_score')

In [ ]:
gs.best_params_, gs.best_score_

In [ ]:
y_ans = gs.predict(X_test)

In [8]:
knn = KNeighborsClassifier(n_neighbors=9, n_jobs=-1)

In [9]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=9, p=2,
           weights='uniform')

In [10]:
y_pred = knn.predict(X_test[0 : 20000])

In [11]:
y_pred.shape

(20000,)

In [12]:
f1_score(y_test[0 : 20000], y_pred)

0.181284834262247

In [161]:
df_test = pd.read_csv('test.csv', sep=',')

In [164]:
col_index = ['pdb_chain' , 'DSSR']
encoded_data_test, encoders_test = number_encode_features(df_test, col_index)
encoded_data_test.head()

,Id,index,pdb_chain,DSSR,xray,resol,chainlen,protein,alpham2,betam2,...,atomO2,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3
0,0,28,1,356,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,0,0,0,1,0,0
1,1,29,1,356,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,0,0,0,0,1,0
2,2,30,1,356,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,0,1,0,0,0,0
3,3,31,1,356,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,1,0,0,0,0,0
4,4,32,1,356,1,2.3,21,1,-73.3,174.9,...,0,0,1,0,0,0,0,0,0,0


In [165]:
encoded_data_test = scale(encoded_data_test.as_matrix())

In [166]:
encoded_data_test.shape

(4045, 387)

In [167]:
y_ans = knn.predict(encoded_data_test[:, 2: ])

In [23]:
def write_ans(preds):
    answer = pd.DataFrame({'Id': np.arange(preds.shape[0])})
    print(len(answer))
    answer['mg'] = preds
    print(len(preds))
    answer.to_csv('ans_catboost_600.csv', index=False)

In [62]:
# Importing Classifier Modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

In [9]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

In [10]:
clf = RandomForestClassifier(n_estimators=50)
scoring = 'f1'
score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
print(score)

[0.5790499  0.57688835 0.57943925 0.57451598 0.55951843]


In [12]:
clf = RandomForestClassifier(n_estimators=100)
scoring = 'f1'
score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
print(score)

[0.58201374 0.57551143 0.58188472 0.57629652 0.5706611 ]


In [13]:
clf = RandomForestClassifier(n_estimators=500)
scoring = 'f1'
score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
print(score)

[0.58832247 0.58275965 0.58848549 0.58072217 0.57518853]


In [11]:
clf = GaussianNB()
scoring = 'f1'
score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
print(score)

[0.25960031 0.25950653 0.2531977  0.25124626 0.2663898 ]


In [60]:
clf = xgboost.XGBClassifier()
scoring = 'f1'
score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
print(score)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

[0.01033325 0.00312012 0.00742312 0.01155462 0.00871571]


In [63]:
clf = AdaBoostClassifier()
scoring = 'f1'
score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
print(score)

[0.02190525 0.02394904 0.03254132 0.03234917 0.03299175]


In [14]:
df_test = pd.read_csv('test.csv', sep=',')

In [15]:
df_test.dropna(inplace=True)
col_index = ['pdb_chain' , 'DSSR']
encoded_data_test, encoders_test = number_encode_features(df_test, col_index)
encoded_data_test.head()

,Id,index,pdb_chain,DSSR,xray,resol,chainlen,protein,alpham2,betam2,...,atomO2,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3
0,0,28,1,356,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,0,0,0,1,0,0
1,1,29,1,356,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,0,0,0,0,1,0
2,2,30,1,356,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,0,1,0,0,0,0
3,3,31,1,356,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,1,0,0,0,0,0
4,4,32,1,356,1,2.3,21,1,-73.3,174.9,...,0,0,1,0,0,0,0,0,0,0


In [75]:
clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
clf.fit(X_train, y_train)

prediction = clf.predict(X_test)

In [76]:
f1_score(y_test, prediction)

0.6043876567020251

In [77]:
y_ans = clf.predict(encoded_data_test.iloc[:, 2: ])

In [78]:
len(prediction)

68985

In [79]:
len(np.argwhere(prediction == 1))

7326

In [64]:
clf_50 = AdaBoostClassifier()
clf_50.fit(X_train, y_train)
prediction = clf_50.predict(X_test)

In [65]:
f1_score(y_test, prediction)

0.028742263715514527

In [66]:
y_ans = clf_50.predict(encoded_data_test.iloc[:, 2: ])

In [80]:
len(np.argwhere(y_ans == 1))

0

In [81]:
len(y_ans)

4045

In [69]:
write_ans(y_ans)

4045
4045


In [16]:
clf_catboost = CatBoostClassifier(iterations=900, metric_period=2, eval_metric='F1', verbose=True, learning_rate=0.3)

In [17]:
clf_catboost.fit(X_train, y_train)

0:	learn: 0.0090257	total: 315ms	remaining: 3m 8s
2:	learn: 0.0017611	total: 923ms	remaining: 3m 3s
4:	learn: 0.0007752	total: 1.55s	remaining: 3m 5s
6:	learn: 0.0026051	total: 2.04s	remaining: 2m 53s
8:	learn: 0.0028160	total: 2.58s	remaining: 2m 49s
10:	learn: 0.0034483	total: 3.14s	remaining: 2m 48s
12:	learn: 0.0085557	total: 3.68s	remaining: 2m 46s
14:	learn: 0.0115538	total: 4.2s	remaining: 2m 43s
16:	learn: 0.0140500	total: 4.74s	remaining: 2m 42s
18:	learn: 0.0164747	total: 5.35s	remaining: 2m 43s
20:	learn: 0.0186076	total: 5.94s	remaining: 2m 43s
22:	learn: 0.0217716	total: 6.49s	remaining: 2m 42s
24:	learn: 0.0227328	total: 7.05s	remaining: 2m 42s
26:	learn: 0.0262291	total: 7.87s	remaining: 2m 47s
28:	learn: 0.0328673	total: 8.4s	remaining: 2m 45s
30:	learn: 0.0364438	total: 8.98s	remaining: 2m 44s
32:	learn: 0.0430322	total: 9.54s	remaining: 2m 44s
34:	learn: 0.0463492	total: 10.1s	remaining: 2m 42s
36:	learn: 0.0516530	total: 10.7s	remaining: 2m 42s
38:	learn: 0.0579001	t

312:	learn: 0.4700513	total: 1m 36s	remaining: 1m 28s
314:	learn: 0.4711967	total: 1m 37s	remaining: 1m 28s
316:	learn: 0.4726930	total: 1m 38s	remaining: 1m 27s
318:	learn: 0.4758255	total: 1m 38s	remaining: 1m 26s
320:	learn: 0.4770507	total: 1m 39s	remaining: 1m 26s
322:	learn: 0.4790559	total: 1m 39s	remaining: 1m 25s
324:	learn: 0.4809282	total: 1m 40s	remaining: 1m 24s
326:	learn: 0.4828752	total: 1m 40s	remaining: 1m 24s
328:	learn: 0.4834738	total: 1m 41s	remaining: 1m 23s
330:	learn: 0.4858057	total: 1m 41s	remaining: 1m 22s
332:	learn: 0.4868593	total: 1m 42s	remaining: 1m 22s
334:	learn: 0.4882733	total: 1m 43s	remaining: 1m 21s
336:	learn: 0.4891848	total: 1m 43s	remaining: 1m 20s
338:	learn: 0.4902866	total: 1m 44s	remaining: 1m 20s
340:	learn: 0.4918419	total: 1m 44s	remaining: 1m 19s
342:	learn: 0.4937783	total: 1m 45s	remaining: 1m 19s
344:	learn: 0.4944568	total: 1m 46s	remaining: 1m 18s
346:	learn: 0.4968019	total: 1m 46s	remaining: 1m 17s
348:	learn: 0.4992888	total:

In [18]:
df_test = pd.read_csv('test.csv', sep=',')
df_test.dropna(inplace=True)
col_index = ['pdb_chain' , 'DSSR']
encoded_data_test, encoders_test = number_encode_features(df_test, col_index)
encoded_data_test = scale(encoded_data_test.as_matrix())

In [20]:
y_catboost_pred = clf_catboost.predict(encoded_data_test[:, 2: ])

In [22]:
len(np.argwhere(y_catboost_pred == 1))

180

In [28]:
write_ans(y_catboost_pred.astype(int))

4045
4045


In [27]:
y_catboost_pred.astype(int)

array([0, 0, 0, ..., 0, 0, 0])